In [7]:
# ------------------------------------------------------------------------------------------------
# Paquetes.
# ------------------------------------------------------------------------------------------------

import pandas as pd
import numpy as np
import math 
import openpyxl
from openpyxl import Workbook, load_workbook
from openpyxl.utils import get_column_letter
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import NamedStyle, Font, PatternFill, Border, Side, Alignment, Color
from openpyxl.worksheet.dimensions import ColumnDimension, DimensionHolder
from openpyxl.worksheet.worksheet import Worksheet


In [8]:

import sys
sys.path.append('c:/Users/tomas/Documents/Programación/Proyectos/Paquetes')
import Forpy as fg
import Excpy as ex
import Frampy as fr
import Stringpy as st


In [3]:

# ------------------------------------------------------------------------------------------------
# Variables.
# ------------------------------------------------------------------------------------------------

# Archivo Proveedores.
Proveedores = 'J:/My Drive/Forraje/Proveedores.xlsx'
df_Proveedores = pd.read_excel(Proveedores, sheet_name='Centro')

# Proveedores.
Lista_Proveedores = ['Benavídez', 'Gallo', 'Anea', 'Pilar', 'Tortuguitas', 'Del Viso',
                     'Chica Glade', 'San Martín', 'Hipocampus', 'Mundo Mascotas', 'Yaguar',
                     'King Clor', 'Diserquim', 'Moreno', 'Roberto', 'Viejo Gets', 'Viejo Cuero', 'Mis Nietos', 
                     'Profesional Vet', 'Premier', 'Dacam']

# Archivo Nex.
Nex = 'J:/My Drive/Forraje/Exportar.xls'
df_Nex = pd.read_excel(Nex)

# Archivo Granel.
Granel = 'J:/My Drive/Forraje/Granel.xlsx'
df_Granel = pd.read_excel(Granel)

# Archivo Fraccionados.
Fraccion = 'J:/My Drive/Forraje/Fraccionables.xls'
df_Fraccion = pd.read_excel(Fraccion)

# Ofertas y precios establecidos.
Ofertas = {}


In [ ]:
df_Proveedores[df_Proveedores['Descripcion'].str.contains('Pato')]

,Descripcion,Codigo,Categoria,Benavídez,Gallo,Anea,Pilar,Tortuguitas,Del Viso,Chica Glade,...,Moreno,Roberto,Viejo Gets,Viejo Cuero,Mis Nietos,Profesional Vet,Premier,Dacam,Proveedor,Saphirus
1406,Pato Adhesivo,1905876.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,1076.67681,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1407,Pato Bloque Mochila x 1 u.,1905841.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,1702.40313,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1408,"Pato Canasta Activa x 38,6 grs.",1906068.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1409,Pato Canasta Liquida Full,1905639.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,2682.97095,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1410,Pato Canasta Liquida Repuesto,1905877.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,2293.58832,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1411,Pato Canasta Solida Full,1906068.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1412,Pato Desinfectante Inodoro Quita Sarro x 500 cc.,1906068.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1413,Pato Disco Adhesivo Full,1905878.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,3709.38219,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1414,Pato Discos Adhesivo Repuesto,1905879.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,2427.00891,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1415,Pato Limpiador Inodoro Gel Durazno x 500 cc.,1906006.0,Limpieza,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# PROCESAMIENTO DEL ARCHIVO PROVEEDORES.XLSX
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------------------
# Formatear el texto de las columnas de proveedores.
# ------------------------------------------------------------------------------------------------

df_Proveedores = fg.Column_Provider_Processing(df_Proveedores, Lista_Proveedores)


In [6]:
df_Proveedores[df_Proveedores['Descripcion'].str.contains('Pato')]

,Descripcion,Codigo,Categoria,Benavídez,Gallo,Anea,Pilar,Tortuguitas,Del Viso,Chica Glade,...,Moreno,Roberto,Viejo Gets,Viejo Cuero,Mis Nietos,Profesional Vet,Premier,Dacam,Proveedor,Saphirus
1406,Pato Adhesivo,1905876.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,1076.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1407,Pato Bloque Mochila x 1 u.,1905841.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,1702.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1408,"Pato Canasta Activa x 38,6 grs.",1906068.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1409,Pato Canasta Liquida Full,1905639.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,2682.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1410,Pato Canasta Liquida Repuesto,1905877.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,2293.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1411,Pato Canasta Solida Full,1906068.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1412,Pato Desinfectante Inodoro Quita Sarro x 500 cc.,1906068.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1413,Pato Disco Adhesivo Full,1905878.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,3709.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1414,Pato Discos Adhesivo Repuesto,1905879.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,2427.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1415,Pato Limpiador Inodoro Gel Durazno x 500 cc.,1906006.0,Limpieza,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


In [ ]:


# ------------------------------------------------------------------------------------------------
# Hallar precio mínimo y proveedor.
# ------------------------------------------------------------------------------------------------

Minimo_Sin_Cero = df_Proveedores[Lista_Proveedores].replace(0, np.nan).min(axis=1, skipna=True)
df_Proveedores['Precio'] = Minimo_Sin_Cero

df_Proveedores = fg.Find_Best_Provider(df_Proveedores, Lista_Proveedores)



In [ ]:

# ------------------------------------------------------------------------------------------------
# Agregar columnas con los precios y proveedores más baratos.
# ------------------------------------------------------------------------------------------------

Columns = ['Codigo', 'Descripcion', 'Precio', 'Categoria', 'Proveedor']

df_Proveedores = df_Proveedores[Columns].copy()
df_Proveedores.rename(columns={'Precio': 'Costo'}, inplace=True)



In [ ]:

# ------------------------------------------------------------------------------------------------
# Agregar columnas con Markups y tipo de Unidad.
# ------------------------------------------------------------------------------------------------

df_Proveedores['Markup %'] = df_Proveedores['Categoria'].map(lambda x: fg.Markup_And_Unity[x]['Markup %'])
df_Proveedores['Unidad'] = df_Proveedores['Categoria'].map(lambda x: fg.Markup_And_Unity[x]['Unidad'])



In [ ]:

# ------------------------------------------------------------------------------------------------
# Filtrar df.
# ------------------------------------------------------------------------------------------------

Columns = ['Descripcion','Costo','Markup %','Proveedor','Categoria', 'Unidad']
df_Proveedores = df_Proveedores[Columns]



In [ ]:


# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# PROCESAMIENTO DEL ARCHIVO EXPORTAR.XLS, CATÁLOGO DEL SISTEMA NEXTAR
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------------------
# Renombrar columnas y filtrar df.
# ------------------------------------------------------------------------------------------------

df_Nex.rename(columns={'Costo Unitário': 'Costo'}, inplace=True)

df_Nex = fr.Add_Word_To_Name_Columns(df_Nex, Word = "Nex", Separator = '_')


In [ ]:

# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# DATAFRAME FINAL.
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

# ------------------------------------------------------------------------------------------------
# Verificar que los dos df tienen el mismo largo y componentes.
# ------------------------------------------------------------------------------------------------

Is_Same_Lenght = fr.Compare_Columns(df_Nex, 'Descripcion_Nex', df_Proveedores, 'Descripcion')

if len(Is_Same_Lenght) != 0:
    print(Is_Same_Lenght)
    raise Exception("Los dfs no tienen la misma longitud o tienen diferencias en los nombres de los productos.")

Columns = ['Codigo_Nex','Costo_Nex','Precio_Nex']
df_Nex = df_Nex[Columns]

df = pd.concat([df_Nex, df_Proveedores], axis=1)


df.rename(columns={'Codigo_Nex': 'Codigo'}, inplace=True)
df.rename(columns={'Costo': 'Costo_Prov'}, inplace=True)
df.rename(columns={'Precio': 'Precio_Nex'}, inplace=True)

Columns_New_Order = ["Codigo", "Descripcion", "Costo Final", "Precio_Nex", "Costo_Nex", "Costo_Prov", "Markup %", "Categoria", "Proveedor"]

df = df.reindex(columns=Columns_New_Order)


In [ ]:

# ------------------------------------------------------------------------------------------------
# Calcular variables nuevas.
# ------------------------------------------------------------------------------------------------

df['Costo Final'] = df.apply(lambda row: max(row['Costo_Nex'], row['Costo_Prov']), axis=1).fillna(0)



In [ ]:

# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# CALCULAR PRECIOS A GRANEL.
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

df_Granel = fr.Get_Last_Number_Of_String_Column(df_Granel, 'Bolsa', 'Kilos')

df_Granel = fr.Match_And_Copy_Column_Values(df_Granel, df, 'Bolsa', 'Costo Bolsa', 'Descripcion',  'Costo Final')

df_Granel = df_Granel.replace('', 0)

df_Granel = fr.Convert_Type_Of_Columns(df_Granel, ['Kilos', 'Costo Bolsa', 'Costo Final'], "float")

df_Granel['Costo Kilo'] = df_Granel['Costo Bolsa']/df_Granel['Kilos']

df = fr.Match_And_Copy_Column_Values(df, df_Granel, 'Descripcion', 'Costo_Prov', 'Granel', 'Costo Kilo')



In [ ]:


# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# CALCULAR PRECIOS DE FRACCIONADOS.
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------
# ------------------------------------------------------------------------------------------------

df_Fraccion = fr.Get_Last_Number_Of_String_Column(df_Fraccion, 'Fraccionable', 'Unidades')

df_Fraccion = fr.Match_And_Copy_Column_Values(df_Fraccion, df, 'Fraccionable', 'Costo Paquete', 'Descripcion', 'Costo Final')

df_Fraccion = df_Fraccion.replace('', 0)

df_Fraccion = fr.Convert_Type_Of_Columns(df_Fraccion, ['Unidades', 'Costo Paquete'], "float")

df_Fraccion['Costo Unidad'] = df_Fraccion['Costo Paquete']/df_Fraccion['Unidades']

In [215]:
df[df['Descripcion'].str.contains('Pato')]

,Codigo,Descripcion,Costo Final,Precio_Nex,Costo_Nex,Costo_Prov,Markup %,Categoria,Proveedor
1406,1905876,Pato Adhesivo,10766768.00,1800.00,1239.00,10766768.00,50,Limpieza,Chica Glade
1407,1905841,Pato Bloque Mochila x 1 u.,17024031.00,3200.00,2180.00,17024031.00,50,Limpieza,Chica Glade
1408,1906365,"Pato Canasta Activa x 38,6 grs.",2859.00,4300.00,2859.00,NaN,50,Limpieza,NaN
1409,1905639,Pato Canasta Liquida Full,26829709.00,4280.00,2950.00,26829709.00,50,Limpieza,Chica Glade
1410,1905877,Pato Canasta Liquida Repuesto,22935883.00,7280.00,5013.00,22935883.00,50,Limpieza,Chica Glade
1411,1905864,Pato Canasta Solida Full,2860.00,4200.00,2860.00,NaN,50,Limpieza,NaN
1412,1905880,Pato Desinfectante Inodoro Quita Sarro x 500 cc.,3190.00,4700.00,3190.00,NaN,50,Limpieza,NaN
1413,1905878,Pato Disco Adhesivo Full,37093821.00,5600.00,3709.00,37093821.00,50,Limpieza,Chica Glade
1414,1905879,Pato Discos Adhesivo Repuesto,24270089.00,4800.00,3270.00,24270089.00,50,Limpieza,Chica Glade
1415,1906006,Pato Limpiador Inodoro Gel Durazno x 500 cc.,2767.20,3990.00,2767.20,NaN,50,Limpieza,NaN


In [153]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Ivomec.
# ------------------------------------------------------------------------------------------------

df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'Ivomec x 500 cc.', 'Costo Unidad'] += df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'Jeringa x 100 u.', 'Costo Unidad'].values[0]

df_Fraccion[df_Fraccion['Fraccionable'] == 'Ivomec x 500 cc.']


,Fraccionable,Unidad,Unidades,Costo Paquete,Costo Unidad
30,Ivomec x 500 cc.,Ivomec Jeringa,500.00,64522.00,282.76


In [154]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: A otra parte.
# ------------------------------------------------------------------------------------------------

# Encontrar la fila donde la columna "Descripcion" es igual a "Envase x 250 cc."
Fila_Botellita = df.loc[df['Descripcion'] == 'Envase x 250 cc.']

# Extraer el valor de la columna "Costo Final" de la fila encontrada
Costo_Botellita = Fila_Botellita['Costo Final'].values[0]

# Utiliza el método .iloc para acceder al primer elemento de la columna y asignarle el nuevo valor
df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'A Otra Parte Concentrado x 600 cc.', 'Costo Unidad'] = df_Fraccion['Costo Unidad'].iloc[0]*25 + Costo_Botellita

df_Fraccion[df_Fraccion['Fraccionable'] == 'A Otra Parte Concentrado x 600 cc.']

,Fraccionable,Unidad,Unidades,Costo Paquete,Costo Unidad
0,A Otra Parte Concentrado x 600 cc.,"A Otra Parte x 0,25 lts.",600.00,9968.00,625.33


In [155]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Bozales.
# ------------------------------------------------------------------------------------------------

Costo = df_Fraccion.loc[df_Fraccion['Fraccionable'] == 'Bozal Canasta Pack x 7 u.', 'Costo Paquete'].values[0]

df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 1', 'Costo Unidad'] = Costo * 0.10
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 2', 'Costo Unidad'] = Costo * 0.12
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 3', 'Costo Unidad'] = Costo * 0.14
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 4', 'Costo Unidad'] = Costo * 0.16
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 5', 'Costo Unidad'] = Costo * 0.20
df_Fraccion.loc[df_Fraccion['Unidad'] == 'Bozal Canasta N° 6', 'Costo Unidad'] = Costo * 0.22

df_Fraccion[df_Fraccion['Fraccionable'] == 'Bozal Canasta Pack x 7 u.']

,Fraccionable,Unidad,Unidades,Costo Paquete,Costo Unidad
31,Bozal Canasta Pack x 7 u.,Bozal Canasta N° 1,7.00,19880.00,1988.00
32,Bozal Canasta Pack x 7 u.,Bozal Canasta N° 2,7.00,19880.00,2385.60
33,Bozal Canasta Pack x 7 u.,Bozal Canasta N° 3,7.00,19880.00,2783.20
34,Bozal Canasta Pack x 7 u.,Bozal Canasta N° 4,7.00,19880.00,3180.80
35,Bozal Canasta Pack x 7 u.,Bozal Canasta N° 5,7.00,19880.00,3976.00
36,Bozal Canasta Pack x 7 u.,Bozal Canasta N° 6,7.00,19880.00,4373.60
37,Bozal Canasta Pack x 7 u.,Bozal Canasta N° 7,7.00,19880.00,2840.00


In [156]:
# Adicionamos los costos por unidad al df.
df = fr.Match_And_Copy_Column_Values(df, df_Fraccion, 'Descripcion', 'Costo_Prov', 'Unidad', 'Costo Unidad')

df[df['Descripcion'].str.contains('Bozal Canasta', case=False, na=False)]

,Codigo,Descripcion,Costo Final,Precio_Nex,Costo_Nex,Costo_Prov,Markup %,Categoria,Proveedor
254,4046,Bozal Canasta N° 1,2840.00,4300.00,2840.00,1988.00,50,Mascotas,Mundo Mascotas
255,4047,Bozal Canasta N° 2,2840.00,4300.00,2840.00,2385.60,50,Mascotas,Mundo Mascotas
256,4048,Bozal Canasta N° 3,3200.00,4800.00,3200.00,2783.20,50,Mascotas,Mundo Mascotas
257,4049,Bozal Canasta N° 4,5680.00,8600.00,5680.00,3180.80,50,Mascotas,Mundo Mascotas
258,4050,Bozal Canasta N° 5,3440.00,4980.00,3181.00,3976.00,50,Mascotas,Mundo Mascotas
259,4051,Bozal Canasta N° 6,4000.00,6000.00,3976.00,4373.60,50,Mascotas,Mundo Mascotas
260,4052,Bozal Canasta N° 7,4400.00,6600.00,4374.00,2840.00,50,Mascotas,Mundo Mascotas
261,1906328,Bozal Canasta Pack x 7 u.,19880.00,29900.00,19880.00,18800.00,50,Mascotas,Hipocampus


In [157]:
# Convertir Markup a float.
df['Markup %'] = df['Markup %'].astype(float)

In [158]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Pulmax Perro.
# ------------------------------------------------------------------------------------------------

Pulmax = ['Pipeta Pulmax 02-05 kg.', 'Pipeta Pulmax 05-10 kg.', 'Pipeta Pulmax 10-20 kg.', 
          'Pipeta Pulmax 20-40 kg.', 'Pipeta Pulmax 40-60 kg.']
    
Contador = 50
for Pipeta in Pulmax:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

In [159]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Pulmax Gato.
# ------------------------------------------------------------------------------------------------

Pulmax_Gato = ['Pipeta Pulmax Gato 00-04 kg.', 'Pipeta Pulmax Gato 04-08 kg.']

Contador = 50 
for Pipeta in Pulmax_Gato:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

df[df['Descripcion'].str.contains('Pipeta Pulmax', case=False, na=False)]

,Codigo,Descripcion,Costo Final,Precio_Nex,Costo_Nex,Costo_Prov,Markup %,Categoria,Proveedor
1532,1905182,Pipeta Pulmax 02-05 kg.,1120.00,1680.00,1120.00,1120.00,50.00,Veterinaria,Tortuguitas
1533,1905183,Pipeta Pulmax 05-10 kg.,1120.00,1800.00,1120.00,1120.00,55.00,Veterinaria,Tortuguitas
1534,1905184,Pipeta Pulmax 10-20 kg.,1180.00,1900.00,1180.00,1180.00,60.00,Veterinaria,Hipocampus
1535,1905185,Pipeta Pulmax 20-40 kg.,1180.00,2000.00,1180.00,1180.00,65.00,Veterinaria,Hipocampus
1536,1905186,Pipeta Pulmax 40-60 kg.,1180.00,2100.00,1180.00,1180.00,70.00,Veterinaria,Hipocampus
1537,1905180,Pipeta Pulmax Gato 00-04 kg.,1180.00,1800.00,1180.00,1180.00,50.00,Veterinaria,Hipocampus
1538,1905181,Pipeta Pulmax Gato 04-08 kg.,1120.00,1900.00,1053.00,1120.00,55.00,Veterinaria,Tortuguitas


In [160]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Hectopar Perro.
# ------------------------------------------------------------------------------------------------

Hectopar = ['Pipeta Hectopar 00-04 kg.', 'Pipeta Hectopar 05-10 kg.', 'Pipeta Hectopar 10-25 kg.',
            'Pipeta Hectopar 25-40 kg.', 'Pipeta Hectopar 40-60 kg.']

Contador = 50 
for Pipeta in Hectopar:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

In [161]:
# ------------------------------------------------------------------------------------------------
# Casos especiales: Hectopar Gato.
# ------------------------------------------------------------------------------------------------

# Hectopar Gato.
Hectopar_Gato = ['Pipeta Hectopar Gato 00-04 kg.', 'Pipeta Hectopar Gato 05-08 kg.']

Contador = 50 
for Pipeta in Hectopar_Gato:
    df.loc[df['Descripcion'] == Pipeta, 'Markup %'] = Contador
    Contador = Contador + 5

df[df['Descripcion'].str.contains('Pipeta Hectopar', case=False, na=False)]

,Codigo,Descripcion,Costo Final,Precio_Nex,Costo_Nex,Costo_Prov,Markup %,Categoria,Proveedor
1516,4257,Pipeta Hectopar 00-04 kg.,1134.00,1700.00,970.00,1134.00,50.00,Veterinaria,Hipocampus
1517,4258,Pipeta Hectopar 05-10 kg.,1134.00,1700.00,790.00,1134.00,55.00,Veterinaria,Hipocampus
1518,4259,Pipeta Hectopar 10-25 kg.,1134.00,1700.00,790.00,1134.00,60.00,Veterinaria,Hipocampus
1519,4260,Pipeta Hectopar 25-40 kg.,1134.00,1700.00,790.00,1134.00,65.00,Veterinaria,Hipocampus
1520,4261,Pipeta Hectopar 40-60 kg.,1219.00,1700.00,1219.00,1134.00,70.00,Veterinaria,Hipocampus
1521,4262,Pipeta Hectopar Gato 00-04 kg.,1219.00,1700.00,1219.00,1134.00,50.00,Veterinaria,Hipocampus
1522,4263,Pipeta Hectopar Gato 05-08 kg.,1219.00,1700.00,1219.00,1134.00,55.00,Veterinaria,Hipocampus


In [162]:
# Columna Precio Proveedores.

df['Precio_Prov'] = df['Costo_Prov']* (1 + (df['Markup %']/100))

In [163]:
# Llenamos df de 0 si hay NaN.
df = df.fillna(0)

In [164]:
# Columna de Relaciones.

df['Precio_Dif'] = df['Precio_Prov'] / df['Precio_Nex']
df['Costo_Dif'] = df['Costo_Prov'] / df['Costo_Nex']

In [165]:
# Columna Costo Final.
df['Costo_Fin'] = df.apply(lambda row: max(row['Costo_Nex'], row['Costo_Prov']), axis=1)

In [166]:
# Columna Precio Final.
df['Precio_Fin'] = df.apply(lambda row: max(row['Precio_Nex'], row['Precio_Prov']), axis=1)

In [167]:
# Redondeo de Precio Final.
df['Precio_Fin'] = df['Precio_Fin'].apply(lambda x: math.ceil(x / 10) * 10)

In [168]:
# Columna Markup Final.
df['Markup_Fin'] = (df['Precio_Fin']/df['Costo_Fin']) - 1

In [169]:
# Columna Markup.
df['Markup %'] = df['Markup %']/100

# Arreglamos precios de ofertas.
for i in range(0, len(df)):

    Producto = df['Descripcion'][i]
    for j in range(0, len(Ofertas)):
        if Producto in Ofertas:
            df.at[i, 'Precio Final'] = Ofertas[Producto]
            
df[df['Descripcion'] == 'Solo x 15 kg.']

,Codigo,Descripcion,Costo Final,Precio_Nex,Costo_Nex,Costo_Prov,Markup %,Categoria,Proveedor,Precio_Prov,Precio_Dif,Costo_Dif,Costo_Fin,Precio_Fin,Markup_Fin
1897,1905982,Solo x 15 kg.,8232.00,10900.00,8232.00,7448.00,0.32,Perro Adulto,Mundo Mascotas,9831.36,0.90,0.90,8232.00,10900,0.32


In [170]:
# Ordenar columnas.
Orden = ['Codigo', 'Descripcion', 'Precio_Fin', 'Costo_Fin', 'Markup_Fin',
         'Costo_Nex', 'Costo_Prov', 'Precio_Nex', 'Precio_Prov', 
         'Precio_Dif','Costo_Dif', 
         'Markup %', 'Proveedor', 'Categoria']

In [171]:
# Reasigna el DataFrame con el nuevo orden de columnas
df = df[Orden]

In [172]:
# Configurar la opción de Pandas para mostrar dos decimales
pd.options.display.float_format = '{:.2f}'.format

In [173]:
# Database entero.

# Guardar archivo.
Ruta = 'J:/My Drive/Forraje/Resultados/Resultados.xlsx'

# Guardar
df.to_excel(Ruta, index=False) 

In [174]:
# Cargamos Excel.
Libro = load_workbook(Ruta) 

In [175]:
# Seleccionar hoja activa.
Hoja = Libro.active 

In [176]:
# Anchos.
Hoja.column_dimensions["A"].width = 15 
Hoja.column_dimensions["B"].width = 40
Hoja.column_dimensions["C"].width = 15 
Hoja.column_dimensions["D"].width = 15 
Hoja.column_dimensions["E"].width = 15 
Hoja.column_dimensions["F"].width = 15 
Hoja.column_dimensions["G"].width = 15 
Hoja.column_dimensions["H"].width = 15 
Hoja.column_dimensions["I"].width = 15 
Hoja.column_dimensions["J"].width = 15 
Hoja.column_dimensions["K"].width = 15 
Hoja.column_dimensions["L"].width = 15 
Hoja.column_dimensions["M"].width = 20 
Hoja.column_dimensions["N"].width = 20 

In [177]:
# Inmovilizar paneles
Hoja.freeze_panes = 'C2'

In [178]:
# Formato $ con dos decimales para columnas de precios y costos.
Estilo = NamedStyle(name="currency")

In [179]:
# Decimales.
Estilo.number_format = '$#,##0'

# Columnas.
Columnas_con_Peso = ["F", "G", "H", "I"]

for Columna in Columnas_con_Peso:
    for Celda in Hoja[Columna]:
        Celda.style = Estilo

In [180]:
# Formato general con dos decimales para columnas de relación.
General = NamedStyle(name="general")

# Decimales.
General.number_format = '0.00'

# Columnas.
for Columna in ["J", "K"]:
    for Celda in Hoja[Columna]:
        Celda.style = General

In [181]:
# Formato general con porcentaje y sin decimales para columna de markup.
General.number_format = '0%'

# Columnas.
for Columna in ["E", "L"]:
    for Celda in Hoja[Columna]:
        Celda.style = General

In [182]:
# Formato general con cero decimales para columnas de precio final y costo final.
for Columna in ["C", "D"]:
    for Celda in Hoja[Columna]:
        Celda.number_format = '0'

In [183]:
# Centrar.
Centrar = Alignment(horizontal='center', vertical='center')

# Iteración sobre cada celda de la fila.

for i in Hoja.columns:
    for Celda in i:
        Celda.alignment = Centrar

In [184]:
# Duplica la hoja 2 veces.
Hoja2_Nombre = "Hoja2"
Hoja2 = Libro.copy_worksheet(Hoja)
Hoja2.title = Hoja2_Nombre

Hoja3_Nombre = "Hoja3"
Hoja3 = Libro.copy_worksheet(Hoja)
Hoja3.title = Hoja3_Nombre 

In [185]:
# Formato condicional de color rojo en Costo Final para comparación Costo Final - Costo Sistema anterior.
for Fila in Hoja.iter_rows(min_row=2, min_col=4, max_row=Hoja.max_row, max_col=6):  
    for Celda1, Celda2 in zip(Fila, Fila[2:]):
        if abs(Celda1.value - Celda2.value) < 5:
            None
        else:
            Celda1.fill = PatternFill(start_color="FF6666", end_color="FF6666", fill_type="solid") 

In [186]:
# Formato condicional de color verde en Precio Final para comparación Precio Final - Precio Sistema.
for Fila in Hoja.iter_rows(min_row=2, min_col=3, max_row=Hoja.max_row, max_col=8):  
    for Celda1, Celda2 in zip(Fila, Fila[5:]):
        if abs(Celda1.value - Celda2.value) < 21:
            None
        else:
            Celda1.fill = PatternFill(start_color="89ac76", end_color="89ac76", fill_type="solid")
            Celda1.font = Font(bold=True) 

In [187]:
# Formato condicional de color azul en Costo Proveedores para comparación Costo Final - Costo Proveedores (cuando este sea bajo).
for Fila in Hoja2.iter_rows(min_row=2, min_col=4, max_row=Hoja2.max_row-1, max_col=7):  
    for Celda1, Celda2 in zip(Fila, Fila[3:]):
        if abs(Celda1.value - Celda2.value) > 21:
            if Celda2.value == 0:
                None
            else:
                Celda2.fill = PatternFill(start_color="B0E0E6", end_color="B0E0E6", fill_type="solid")
                Celda2.font = Font(bold=True) 

In [188]:
# Formato condicional de color naranja en Costo Final para comparación Costo Final - Costo Proveedores (cuando este sea alto).
for Fila in Hoja3.iter_rows(min_row=2, min_col=4, max_row=Hoja3.max_row-1, max_col=7):  
    for Celda1, Celda2 in zip(Fila, Fila[3:]):
        if Celda1.value - Celda2.value < 0:
            if Celda2.value == 0:
                None
            else:
                Celda1.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")
                Celda1.font = Font(bold=True) 

In [189]:
# Formato de filtro a la primera fila de la Hoja 1.
Hoja.auto_filter.ref = Hoja.dimensions

In [190]:
# Obtener las filas que tienen formato condicional aplicado.
Filas_Negrita = set()
for Fila in Hoja.iter_rows(min_row=2, max_row=Hoja.max_row, min_col=3, max_col=3):  
    for Celda in Fila:
        if Celda.font == Font(bold=True): 
            Filas_Negrita.add(Fila[0].row)

In [191]:
# Ocultar todas las filas.
for i in range(2, Hoja.max_row + 1):  # Empezar desde la fila 2 ya que la fila 1 contiene los encabezados
    Hoja.row_dimensions[i].hidden = True


In [192]:
# Mostrar solo las filas con formato condicional aplicado.
for Fila in Filas_Negrita:
    Hoja.row_dimensions[Fila].hidden = False

In [193]:
# Contar el número de celdas con negrita.
Negrita = len(Filas_Negrita)

In [194]:
# Escribir el número de celdas con negrita en la fila siguiente a la última fila del documento
Fila_Ultima = Hoja.max_row + 1
Celda_Texto = Hoja.cell(row=Fila_Ultima, column=3)
Celda_Texto.value = "Productos:"
Celda_Texto.alignment = Alignment(horizontal="center")
Celda_Resultado = Hoja.cell(row=Fila_Ultima, column=4)
Celda_Resultado.value = Negrita
Celda_Resultado.alignment = Alignment(horizontal="center") 

In [195]:
# Formato de filtro a la primera fila de la Hoja 2 y 3.
Hoja2.auto_filter.ref = Hoja2.dimensions
Hoja3.auto_filter.ref = Hoja3.dimensions

In [196]:
# Obtener las filas que tienen formato condicional aplicado en Hoja 2 y 3.
Filas_Negrita_2 = set()
for Fila in Hoja2.iter_rows(min_row=2, max_row=Hoja2.max_row, min_col=7, max_col=7):  
    for Celda in Fila:
        if Celda.font == Font(bold=True): 
            Filas_Negrita_2.add(Fila[0].row)

In [197]:
# Ocultar todas las filas en Hoja 2.
for i in range(2, Hoja2.max_row + 1):  # Empezar desde la fila 2 ya que la fila 1 contiene los encabezados
    Hoja2.row_dimensions[i].hidden = True

In [198]:
# Mostrar solo las filas con formato condicional aplicado en Hoja 2.
for Fila in Filas_Negrita_2:
    Hoja2.row_dimensions[Fila].hidden = False

In [199]:
# Lo mismo para la Hoja 3. 
    
Filas_Negrita_3 = set()
for Fila in Hoja3.iter_rows(min_row=2, max_row=Hoja3.max_row, min_col=7, max_col=7):  
    for Celda in Fila:
        if Celda.font == Font(bold=True): 
            Filas_Negrita_3.add(Fila[0].row)
            
for i in range(2, Hoja3.max_row + 1):  # Empezar desde la fila 2 ya que la fila 1 contiene los encabezados
    Hoja3.row_dimensions[i].hidden = True           
            
for Fila in Filas_Negrita_3:
    Hoja3.row_dimensions[Fila].hidden = False

In [200]:
# Inmovilizar paneles en Hoja2 y 3.
Hoja2.freeze_panes = 'C2'
Hoja3.freeze_panes = 'C2'

In [201]:
# Calcula el número de filas en la Hoja2.
Filas = Hoja2.max_row

In [202]:
# Itera sobre todas las filas y mueve los valores de la columna
for i in range(1, Filas):
    # Obtiene el valor de la celda en la columna de origen
    Valor_Celda_Origen = Hoja2.cell(row=i, column=7).value
    
    # Inserta el valor en la nueva posición
    Hoja2.cell(row=i, column=5).value = Valor_Celda_Origen

# Elimina la columna original
Hoja2.delete_cols(7)

In [203]:
# Calcula el número de filas en la Hoja3.
Filas = Hoja3.max_row

In [204]:
# Itera sobre todas las filas y mueve los valores de la columna
for i in range(1, Filas):
    # Obtiene el valor de la celda en la columna de origen
    Valor_Celda_Origen = Hoja3.cell(row=i, column=7).value
    
    # Inserta el valor en la nueva posición
    Hoja3.cell(row=i, column=5).value = Valor_Celda_Origen

In [205]:
# Elimina la columna original
Hoja3.delete_cols(7)

In [206]:
# Formato para las columnas de interés en Hoja2.

# Decimales.
General.number_format = '0'

# Columnas.
for Columna in ["D","E"]:
    for Celda in Hoja2[Columna]:
        Celda.style = General
        Celda.alignment = Centrar
        Celda.fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")
        
for Celda in Hoja2["E"]:
    Celda.fill = PatternFill(start_color="87CEEB", end_color="87CEEB", fill_type="solid")

In [207]:
# Formato para las columnas de interés en Hoja3.

# Decimales.
General.number_format = '0'

# Columnas.
for Columna in ["D","E"]:
    for Celda in Hoja3[Columna]:
        Celda.style = General
        Celda.alignment = Centrar
        Celda.fill = PatternFill(start_color="808080", end_color="808080", fill_type="solid")
        
for Celda in Hoja3["E"]:
    Celda.fill = PatternFill(start_color="FFA500", end_color="FFA500", fill_type="solid")

In [208]:
# Cambiar nombres de hojas.
Hoja.title = 'Precio Calculado Alto'
Hoja2.title = 'Costo Calculado Bajo'
Hoja3.title = 'Costo Calculado Alto' 

In [209]:
# Guardar el archivo
Libro.save(Ruta)

In [210]:
# Abrir libro.

In [211]:
# Abrir Nex.

In [212]:
# Empezar a pasar precios.

In [213]:
# Documento de Baratos.